# 🔄 Bronze to Silver - Risk, Compliance & Audit

Ce notebook transforme les données brutes (bronze) en tables Delta nettoyées (silver) dans le Lakehouse.

## Étapes :
1. Charger les fichiers CSV depuis Files/bronze/
2. Appliquer les transformations (typage des dates, nettoyage)
3. Écrire les tables Delta dans le Lakehouse

In [ ]:
# Cell 1: Load Bronze data
print("📥 Chargement des données bronze...")

controls = spark.read.csv("Files/bronze/controls.csv", header=True, inferSchema=True)
executions = spark.read.csv("Files/bronze/control_executions.csv", header=True, inferSchema=True)
incidents = spark.read.csv("Files/bronze/incidents.csv", header=True, inferSchema=True)
remediation = spark.read.csv("Files/bronze/remediation_actions.csv", header=True, inferSchema=True)
vendors = spark.read.csv("Files/bronze/vendors.csv", header=True, inferSchema=True)

print(f"✅ Controls: {controls.count()} lignes")
print(f"✅ Executions: {executions.count()} lignes")
print(f"✅ Incidents: {incidents.count()} lignes")
print(f"✅ Remediation: {remediation.count()} lignes")
print(f"✅ Vendors: {vendors.count()} lignes")

In [ ]:
# Cell 2: Transform to Silver (typage, nettoyage)
from pyspark.sql.functions import col, to_date

print("🔧 Transformation des données...")

# Controls - déjà propre
controls_silver = controls

# Control Executions - typage date
executions_silver = executions \
    .withColumn("execution_date", to_date(col("execution_date")))

# Incidents - typage date
incidents_silver = incidents \
    .withColumn("detection_date", to_date(col("detection_date")))

# Remediation Actions - typage dates multiples
remediation_silver = remediation \
    .withColumn("start_date", to_date(col("start_date"))) \
    .withColumn("target_completion_date", to_date(col("target_completion_date"))) \
    .withColumn("completion_date", to_date(col("completion_date")))

# Vendors - typage date et risk_score
vendors_silver = vendors \
    .withColumn("last_audit_date", to_date(col("last_audit_date"))) \
    .withColumn("risk_score", col("risk_score").cast("float"))

print("✅ Transformations appliquées!")

In [ ]:
# Cell 3: Write to Delta Tables (Silver)
print("💾 Écriture des tables Delta...")

controls_silver.write.mode("overwrite").format("delta").saveAsTable("controls")
print("✅ Table 'controls' créée")

executions_silver.write.mode("overwrite").format("delta").saveAsTable("control_executions")
print("✅ Table 'control_executions' créée")

incidents_silver.write.mode("overwrite").format("delta").saveAsTable("incidents")
print("✅ Table 'incidents' créée")

remediation_silver.write.mode("overwrite").format("delta").saveAsTable("remediation_actions")
print("✅ Table 'remediation_actions' créée")

vendors_silver.write.mode("overwrite").format("delta").saveAsTable("vendors")
print("✅ Table 'vendors' créée")

print("\n🎉 Silver tables created successfully!")
print("📊 Vérifiez les tables dans la section 'Tables' du Lakehouse")

In [ ]:
# Cell 4: Validation - Aperçu des tables créées
print("📊 Aperçu des tables créées:\n")

print("=== CONTROLS ===")
spark.sql("SELECT * FROM controls LIMIT 5").show()

print("\n=== CONTROL EXECUTIONS ===")
spark.sql("SELECT * FROM control_executions LIMIT 5").show()

print("\n=== INCIDENTS ===")
spark.sql("SELECT * FROM incidents LIMIT 5").show()

print("\n=== REMEDIATION ACTIONS ===")
spark.sql("SELECT * FROM remediation_actions LIMIT 5").show()

print("\n=== VENDORS ===")
spark.sql("SELECT * FROM vendors LIMIT 5").show()